In [42]:
file_dict = {'amazon' : "/content/amazon_cells_labelled.txt",
"imdb" : "/content/imdb_labelled.txt"  ,
"yelp" : "/content/yelp_labelled.txt"  
}

In [43]:
import pandas as pd


In [44]:
df_list = []
for source , path in file_dict.items():
  df = pd.read_csv(path , sep = '\t', names=["reviews","label"])
  df['source']=source
  print(df.shape)
  df_list.append(df)
data = pd.concat(df_list)

(1000, 3)
(748, 3)
(1000, 3)


In [45]:
data.shape

(2748, 3)

In [46]:
data.head()

,reviews,label,source
0,So there is no way for me to plug it in here i...,0,amazon
1,"Good case, Excellent value.",1,amazon
2,Great for the jawbone.,1,amazon
3,Tied to charger for conversations lasting more...,0,amazon
4,The mic is great.,1,amazon


In [47]:
sentences = ['John likes ice cream', 'John hates chocolate.', "John play football"]

In [48]:
from sklearn.feature_extraction.text import CountVectorizer

vec = CountVectorizer(min_df = 0, lowercase=False)
vec.fit(sentences )
print(vec.vocabulary_)

{'John': 0, 'likes': 6, 'ice': 5, 'cream': 2, 'hates': 4, 'chocolate': 1, 'play': 7, 'football': 3}


In [49]:
vec.transform(sentences).toarray()

array([[1, 0, 1, 0, 0, 1, 1, 0],
       [1, 1, 0, 0, 1, 0, 0, 0],
       [1, 0, 0, 1, 0, 0, 0, 1]])

In [50]:
from sklearn.model_selection import train_test_split
df_yelp = data[data["source"]=="yelp"]

sentences = df_yelp["reviews"].values
y = df_yelp['label'].values

sentences_train, sentences_test, y_train, y_test = train_test_split(
   sentences, y, test_size=0.25, random_state=1000)

In [53]:
list(sentences_train[:10])

['The food was barely lukewarm, so it must have been sitting waiting for the server to bring it out to us.',
 'Sorry, I will not be getting food from here anytime soon :(',
 'Of all the dishes, the salmon was the best, but all were great.',
 'The fries were not hot, and neither was my burger.',
 "In fact I'm going to round up to 4 stars, just because she was so awesome.",
 'Will go back next trip out.',
 'This was my first crawfish experience, and it was delicious!',
 "I could barely stomach the meal, but didn't complain because it was a business lunch.",
 'A great way to finish a great.',
 'Best service and food ever, Maria our server was so good and friendly she made our day.']

In [54]:
list(y_train)[:10]

[0, 0, 1, 0, 1, 1, 1, 0, 1, 1]

In [59]:
from sklearn.feature_extraction.text import CountVectorizer

vect = CountVectorizer()
vect.fit(sentences_train)

Xtrain = vect.transform(sentences_train)
xtest = vect.transform(sentences_test)

Xtrain

<750x1714 sparse matrix of type '<class 'numpy.int64'>'
	with 7368 stored elements in Compressed Sparse Row format>

In [60]:
from numpy.ma.core import logical_and
from sklearn.linear_model import LogisticRegression

lr = LogisticRegression()
lr.fit(Xtrain,y_train)

score = lr.score(xtest,y_test)


In [61]:
score

0.796

In [66]:
for source in data['source'].unique(): # yelp , imdb , amazon
    df_source = data[data['source'] == source]
    sentences = df_source['reviews'].values
    y = df_source['label'].values

    sentences_train, sentences_test, y_train, y_test = train_test_split(
        sentences, y, test_size=0.25, random_state=1000)

    vectorizer = CountVectorizer()
    vectorizer.fit(sentences_train)
    X_train = vectorizer.transform(sentences_train)
    X_test  = vectorizer.transform(sentences_test)

    classifier = LogisticRegression()
    classifier.fit(X_train, y_train)
    score = classifier.score(X_test, y_test)
    print('Accuracy for {} data: {:.4f}'.format(source, score))

Accuracy for amazon data: 0.7960
Accuracy for imdb data: 0.7487
Accuracy for yelp data: 0.7960
